## Support Vector Machines - Word Embeddings model

In [ ]:
out# Import the necessary libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

In [ ]:
# Load the pre-trained GloVe embeddings
glove_file = "glove.6B.200d.txt"
word_embeddings = {}
with open(glove_file, "r", encoding="utf-8") as f:
    for line in f:
        values = line.split()
        word = values[0]
        embedding = np.array(values[1:], dtype="float32")
        word_embeddings[word] = embedding

In [ ]:
# Load the dataset
data = pd.read_csv("balanced.csv")

In [ ]:
# Define variables
X = data['clean_text']
y = data['condition_label']

In [ ]:
# Convert a sentence into a feature vector using GloVe embeddings
def sentence_to_vector(words):
    vectors = [word_embeddings[word] for word in words if word in word_embeddings]
    return np.mean(vectors, axis=0) if vectors else np.zeros(200)

X = np.array([sentence_to_vector(words) for words in X])
y = np.array(y)

In [ ]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=12)

In [ ]:
# Define the hyperparameter grid for SVM
param_grid = {
    'kernel': ['linear', 'rbf', 'sigmoid'],
    'C': [0.1, 1, 10, 100],
    'gamma': ['scale', 'auto', 0.1, 0.01, 0.001],
}

In [ ]:
# Use GridSearchCV to define best hyperparameters
svm_classifier = SVC()

grid_search = GridSearchCV(svm_classifier, param_grid, cv=5, n_jobs=-1, verbose=1)
grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_
best_svm_classifier = grid_search.best_estimator_

print("Best Hyperparameters:", best_params)

In [ ]:
# Make predictions on the test data
y_pred = best_svm_classifier.predict(X_test)

In [ ]:
# Calculate accuracy and print classification report
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.5f}")

In [ ]:
# Print classification report
print(classification_report(y_test, y_pred))

In [ ]:
# Define and display confusion matrix
def plot_confusion_matrix(y_test, y_pred, labels):
    cm = confusion_matrix(y_test, y_pred, labels=labels)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=labels)
    fig, ax = plt.subplots(figsize=(6,6))
    plt.rcParams.update({'font.size':13, 'font.monospace':'Computer Modern Typewriter'})
    disp.plot(ax=ax, cmap = "binary", colorbar=False)

plot_confusion_matrix(y_test, y_pred,   labels=best_svm_classifier.classes_)